In [1]:
from __future__ import division, print_function

In [2]:
# beliefs as gaussians

In [8]:
from collections import namedtuple
gaussian = namedtuple('Gaussian', ['mean', 'var'])
print(gaussian)
print(type(gaussian))

<class '__main__.Gaussian'>
<type 'type'>


In [10]:
g1 = gaussian(3.4, 10.1)
g2 = gaussian(mean=4.5, var=0.2**2)
print(g1)
print(g2)

Gaussian(mean=3.4, var=10.1)
Gaussian(mean=4.5, var=0.04000000000000001)


In [6]:
# understanding gaussian multiplication 

z = gaussian(10., 1.) 

In [11]:
def update(prior, likelihood):
    posterior = gaussian_multiply(likelihood, prior)
    return posterior

In [12]:


def gaussian_multiply(g1, g2):
    mean = (g1.var * g2.mean + g2.var * g1.mean) / (g1.var + g2.var)
    variance = (g1.var * g2.var) / (g1.var + g2.var)
    return gaussian(mean, variance)

In [13]:
def predict(pos, movement):
    return gaussian(pos.mean + movement.mean, pos.var + movement.var)

In [15]:
process_var = 1. # variance in the drones movevment
sensor_var = 2. # variance in the sensor  
#                 for a real Kalman filter of course you will not be randomly changing these values.


zs = [1,2,3,4]  # not good data just put there to move fourwared

x = gaussian(0., 20.**2)  # drones ihnintial position, N(0, 20**2), remember we turned our beliefs into gausisan 
velocity = 1
dt = 1. # time step in seconds
process_model = gaussian(velocity*dt, process_var) # displacement to add to x  # a descrtip of how we think the drone moves 
                                                    # the concrol input ot the robot

# perform Kalman filter
for z in zs:       # is a measurement 
    prior = predict(x, process_model)    # returns prior gausisan 
    likelihood = gaussian(z, sensor_var)
    x = update(prior, likelihood) # returns posterios

In [17]:
# The filtering is implemented in only a few lines of code. Most of the code is either initialization, storing of data, simulating the dog movement, and printing results. The code that performs the filtering is very succinct:

prior = predict(x, process_model)
likelihood = gaussian(z, sensor_var)
x = update(prior, likelihood)

# another way instad of using the predict and update functions
# for z in zs:
#     # predict
#     dx = velocity*dt
#     pos = pos + dx
#     var = var + process_var

#     # update
#     pos  = (var*z + sensor_var*pos) / (var + sensor_var)
#     var = (var * sensor_var) / (var + sensor_var)

In [18]:
# perform Kalman filter
for z in zs:       # is a measurement 
    prior = predict(x, process_model)    # returns prior gausisan 
    likelihood = gaussian(z, sensor_var)
    x = update(prior, likelihood) # returns posterios
    

In [19]:
# The Kalman gain $K$ is a scale factor that chooses a value along the residual. This leads to an alternative but equivalent implementation for update() and predict():

def update(prior, measurement):
    x, P = prior        # mean and variance of prior
    z, R = measurement  # mean and variance of measurement
    
    y = z - x        # residual
    K = P / (P + R)  # Kalman gain

    x = x + K*y      # posterior
    P = (1 - K) * P  # posterior variance
    return gaussian(x, P)

def predict(posterior, movement):
    x, P = posterior # mean and variance of posterior
    dx, Q = movement # mean and variance of movement
    x = x + dx
    P = P + Q
    return gaussian(x, P)

# R measurement noise
# Q process noise
# P variance 

# FULL DESCRIPTION OF THE ALGORITHM
# The Kalman filter makes a prediction,
# takes a measurement, 
# and then forms a new estimate somewhere between the two.

# THE GENERIC ALGORITHM 

Initialization

1. Initialize the state of the filter
2. Initialize our belief in the state

Predict

1. Use system behavior to predict state at the next time step
2. Adjust belief to account for the uncertainty in prediction

Update

1. Get a measurement and associated belief about its accuracy
2. Compute residual between estimated state and measurement
3. Compute scaling factor based on whether the measurement
or prediction is more accurate
4. set state between the prediction and measurement based 
on scaling factor
5. update belief in the state based on how certain we are 
in the measurement



In [3]:
import numpy as np
from numpy import dot
dt = 0.1

F = np.array([[1,dt],
              [0,1]])
print(F)

[[1.  0.1]
 [0.  1. ]]


In [4]:
print(F.T)

[[1.  0. ]
 [0.1 1. ]]


In [5]:
P = np.diag([500, 49])
print(P)

[[500   0]
 [  0  49]]


In [6]:
FP = dot(F,P)
print(FP)

[[500.    4.9]
 [  0.   49. ]]


In [7]:
dot(dot(F,P),F.T)

array([[500.49,   4.9 ],
       [  4.9 ,  49.  ]])

In [8]:
def innovation(x, P, F):
    
    x = dot(F, x)
    P = dot(dot(F,P),F.T)
    return x, P

In [9]:
x = np.array([10.0, 4.5])

x, P = innovation(x=x, P=P, F=F)

In [10]:
print(P)

[[500.49   4.9 ]
 [  4.9   49.  ]]


In [11]:
for _ in range (4):
    x ,P = innovation(x=x, P=P, F=F)
    print('x = ', x)

('x = ', array([10.9,  4.5]))
('x = ', array([11.35,  4.5 ]))
('x = ', array([11.8,  4.5]))
('x = ', array([12.25,  4.5 ]))


In [12]:
print(P)

[[512.25  24.5 ]
 [ 24.5   49.  ]]
